# Etude sur la serie M5 forecasting.
## Prise en main du dataset et analyse
## Modelisation et forecasting
## Selection du meilleure

In [1]:
## Import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from ydata_profiling import ProfileReport

In [2]:
## Import data
calendar = pd.read_csv('data/calendar.csv') # Calendrier
sales_train_validation = pd.read_csv('data/sales_train_validation.csv') # Ventes d'articles
sell_prices = pd.read_csv('data/sell_prices.csv') # Prix de vente
sales_train_evalidation = pd.read_csv('data/sales_train_evaluation.csv') #


In [3]:
## print les shapes
print("Calendar shape:", calendar.shape)
print("Sales train validation shape:", sales_train_validation.shape)
print("Sales train evalidation shape:", sales_train_evalidation.shape)
print("Sell prices shape:", sell_prices.shape)

Calendar shape: (1969, 14)
Sales train validation shape: (30490, 1919)
Sales train evalidation shape: (30490, 1947)
Sell prices shape: (6841121, 4)


In [4]:
## Visualisation des données(head)
sales_train_validation.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [5]:
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [6]:
sell_prices.head(5)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [7]:
## Choisir 10 produit aux hasards (avec leur boutiques,leur prices,leur catégories) pour la suite de l'analyse

np.random.seed(42)
random_indices = np.random.choice(sales_train_validation.shape[0], 10, replace=False)
selected_products = sales_train_validation.iloc[random_indices]

# Extraire les informations des produits sélectionnés
print("Produits sélectionnés:")
print(selected_products[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']])

# Créer un dataframe avec les informations complètes (prix inclus)
products_info = selected_products[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].copy()

# Afficher les produits sélectionnés
print("\n10 produits sélectionnés avec leurs informations:")
print(products_info)


Produits sélectionnés:
                                    id          item_id      dept_id  \
2404       FOODS_3_180_CA_1_validation      FOODS_3_180      FOODS_3   
7576   HOUSEHOLD_2_383_CA_3_validation  HOUSEHOLD_2_383  HOUSEHOLD_2   
8731       FOODS_3_409_CA_3_validation      FOODS_3_409      FOODS_3   
4756       FOODS_1_097_CA_2_validation      FOODS_1_097      FOODS_1   
15509    HOBBIES_1_272_TX_2_validation    HOBBIES_1_272    HOBBIES_1   
9773   HOUSEHOLD_1_064_CA_4_validation  HOUSEHOLD_1_064  HOUSEHOLD_1   
17439      FOODS_2_368_TX_2_validation      FOODS_2_368      FOODS_2   
19244  HOUSEHOLD_1_395_TX_3_validation  HOUSEHOLD_1_395  HOUSEHOLD_1   
1092   HOUSEHOLD_1_537_CA_1_validation  HOUSEHOLD_1_537  HOUSEHOLD_1   
16750  HOUSEHOLD_2_410_TX_2_validation  HOUSEHOLD_2_410  HOUSEHOLD_2   

          cat_id store_id state_id  
2404       FOODS     CA_1       CA  
7576   HOUSEHOLD     CA_3       CA  
8731       FOODS     CA_3       CA  
4756       FOODS     CA_2       CA  

In [8]:
## Sauvegarder les produits sélectionnés dans un fichier CSV pour une analyse ultérieure
products_info.to_csv('selected_products_info.csv', index=False)

In [9]:
## Import data
data_selected = pd.read_csv('data/selected_products_info.csv')
data_selected.head(10)


,id,item_id,dept_id,cat_id,store_id,state_id
0,FOODS_3_180_CA_1_validation,FOODS_3_180,FOODS_3,FOODS,CA_1,CA
1,HOUSEHOLD_2_383_CA_3_validation,HOUSEHOLD_2_383,HOUSEHOLD_2,HOUSEHOLD,CA_3,CA
2,FOODS_3_409_CA_3_validation,FOODS_3_409,FOODS_3,FOODS,CA_3,CA
3,FOODS_1_097_CA_2_validation,FOODS_1_097,FOODS_1,FOODS,CA_2,CA
4,HOBBIES_1_272_TX_2_validation,HOBBIES_1_272,HOBBIES_1,HOBBIES,TX_2,TX
5,HOUSEHOLD_1_064_CA_4_validation,HOUSEHOLD_1_064,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA
6,FOODS_2_368_TX_2_validation,FOODS_2_368,FOODS_2,FOODS,TX_2,TX
7,HOUSEHOLD_1_395_TX_3_validation,HOUSEHOLD_1_395,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX
8,HOUSEHOLD_1_537_CA_1_validation,HOUSEHOLD_1_537,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA
9,HOUSEHOLD_2_410_TX_2_validation,HOUSEHOLD_2_410,HOUSEHOLD_2,HOUSEHOLD,TX_2,TX



# Comprehension de la data
---

## 🧩 Vue d’ensemble

Le but du dataset :

> *Prédire les ventes quotidiennes de chaque produit pour les 28 prochains jours*, à partir de données historiques, de prix et de calendriers promotionnels.

Les données couvrent :

* *plus de 3 000 produits*
* *10 magasins*
* *3 États américains*
* *1 941 jours d’historique* (~ 5 ans)

---

## 📂 Les fichiers CSV et leur rôle

### 1️⃣ sales_train_validation.csv

➡ *Fichier principal* — il contient les *ventes journalières historiques*.

| id                 | item_id       | dept_id   | cat_id  | store_id | state_id | d_1 | d_2 | ... | d_1941 |
| ------------------ | ------------- | --------- | ------- | -------- | -------- | --- | --- | --- | ------ |
| HOBBIES_1_001_CA_1 | HOBBIES_1_001 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 2   | ... | 3      |

* Chaque ligne correspond à *un produit* dans un *magasin*.
* Les colonnes d_1 à d_1941 représentent les *ventes journalières* (nombre d’unités vendues chaque jour).
* Il sert de *base d’entraînement*.

---

### 2️⃣ calendar.csv

➡ *Contient les métadonnées sur les jours* (liens avec d_1, d_2, etc.)

| date       | wm_yr_wk | weekday  | wday | month | year | event_name_1 | event_type_1 | snap_CA | snap_TX | snap_WI |
| ---------- | -------- | -------- | ---- | ----- | ---- | ------------ | ------------ | ------- | ------- | ------- |
| 2011-01-29 | 11101    | Saturday | 1    | 1     | 2011 | NaN          | NaN          | 0       | 0       | 0       |

* Lie d_1 → 2011-01-29, d_2 → 2011-01-30, etc.
* snap_CA/TX/WI : indicateur promotionnel (1 = jour de promotion dans l’État).
* event_name_1 : événements spéciaux (Noël, Thanksgiving, etc.).

---

### 3️⃣ sell_prices.csv

➡ *Prix de vente* des produits par semaine et par magasin.

| store_id | item_id       | wm_yr_wk | sell_price |
| -------- | ------------- | -------- | ---------- |
| CA_1     | HOBBIES_1_001 | 11325    | 9.58       |

* Permet d’introduire la *variabilité des prix* dans le modèle (promotions, inflation, etc.).

---

### 4️⃣ sales_train_evaluation.csv

➡ Identique à sales_train_validation.csv mais avec *28 jours supplémentaires* (jusqu’à d_1969).

* Ce fichier a été ajouté pour l’*évaluation finale* (après la compétition).

---

### 5️⃣ sample_submission.csv

➡ Exemple de format attendu pour la *soumission des prédictions*.

| id                            | F1 | F2 | ... | F28 |
| ----------------------------- | -- | -- | --- | --- |
| HOBBIES_1_001_CA_1_validation | 0  | 0  | ... | 0   |

* Chaque ligne = un produit à prédire sur 28 jours.
* Les colonnes F1–F28 sont les *jours futurs*.

---

## 🧠 Comment utiliser ces fichiers ensemble

### Étapes typiques d’un projet M5 :

1️⃣ *Charger les données*

python
import pandas as pd

sales = pd.read_csv("sales_train_validation.csv")
calendar = pd.read_csv("calendar.csv")
prices = pd.read_csv("sell_prices.csv")


2️⃣ *Relier les fichiers*

* Convertir d_1, d_2, … en dates via calendar.csv.
* Joindre sell_prices pour ajouter les prix correspondants à chaque ligne.

3️⃣ *Créer un dataset enrichi*

* Variables temporelles : jour de la semaine, mois, année, événements.
* Variables de prix : prix actuel, variation du prix.
* Variables de ventes passées (lags, moyennes mobiles, etc.)

---

## 💡 En résumé

| Fichier                      | Contenu                                                | Utilisation                           |
| ---------------------------- | ------------------------------------------------------ | ------------------------------------- |
| sales_train_validation.csv | Historique des ventes journalières                     | Base d’entraînement                   |
| calendar.csv               | Informations sur les jours (dates, événements, promos) | Ajouter contexte temporel             |
| sell_prices.csv            | Prix de vente hebdomadaires                            | Ajouter contexte économique           |
| sales_train_evaluation.csv | Données de test prolongées                             | Évaluation finale                     |
| sample_submission.csv      | Format de sortie attendu                               | Vérifier la structure des prédictions |

---


In [10]:

profile = ProfileReport(data_selected, title="Analyse des produits sélectionnés", explorative=True)
# profile

## Prévision des ventes (Time Series Forecasting)

* - Charger les données et vérifier les shapes
* - Nettoyage et harmonisation des tables (calendar, sales, prices)
* - Sélection et préparation des séries à analyser
* - Génération de features temporelles et catégorielles
* - Construction des jeux de retard (lags) et fenêtres glissantes
* - Division temporelle train / validation / test (rolling split)
* - Baselines rapides (naïf, seasonal naive, moving average)
* - Modèles classiques (ARIMA / SARIMAX, Prophet) — entrainement et prédictions
* - Modèles ML avec features tabulaires (XGBoost / RandomForest)
* - Préparation de données pour DL (tf.data, sequences, batching)
* - Modèles séquentiels (LSTM / GRU / CNN1D + LSTM ) — architecture et entraînement

* - Callbacks, sauvegarde checkpoints et reprise d'entraînement
* - Backtesting / validation temporelle (rolling-origin) et scoring
* - Implémentation des métriques (MAE, RMSE, MAPE, RMSSE)



In [12]:
## Netoyage et harmonisation des tables (calendar,sales,prices)
# Conversion des colonnes d_1 à d_1941 en format long
sales_long = sales_train_validation.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Extraction du numéro du jour (ex: d_1 -> 1)
sales_long['day_num'] = sales_long['d'].str.extract('(\d+)').astype(int)

print(f"Sales long format shape: {sales_long.shape}")
sales_long.head()


<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\0000\AppData\Local\Temp\ipykernel_27280\3710895778.py:10: SyntaxWarning: invalid escape sequence '\d'
  sales_long['day_num'] = sales_long['d'].str.extract('(\d+)').astype(int)


Sales long format shape: (58327370, 9)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,day_num
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1


In [13]:
# On s'assure que calendar a les bons index
calendar['day_num'] = range(1, len(calendar) + 1)

# Fusion avec le calendrier
df = sales_long.merge(
    calendar[['day_num', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 
              'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI']],
    on='day_num',
    how='left'
)

# Conversion de la date
df['date'] = pd.to_datetime(df['date'])

print(f"DataFrame after calendar merge: {df.shape}")
df.head()

DataFrame after calendar merge: (58327370, 19)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,day_num,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0


In [14]:
# Fusion avec les prix (attention : les prix sont par semaine wm_yr_wk)
df = df.merge(
    sell_prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

print(f"DataFrame final shape: {df.shape}")
df.head()

DataFrame final shape: (58327370, 20)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,day_num,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,1,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,NaN


In [ ]:
# Sauvegarder le DataFrame final pour une utilisation ultérieure
# df.to_csv('data/m5_merged_data.csv', index=False)

In [3]:
### Load merged data
# merged_data = pd.read_csv('data/m5_merged_data.csv')


## Preparation de la data pour le DL


In [ ]:
# Option 2: Augmenter la période temporelle (12 mois au lieu de 6)
def create_lightweight_dataset(df, sample_fraction=0.1, max_products=100, days_back=365):
    """
    Crée un dataset allégé pour le développement
    """
    # Sélectionner un sous-ensemble de produits
    all_products = df['id'].unique()
    
    if len(all_products) > max_products:
        selected_products = np.random.choice(
            all_products, 
            size=max_products, 
            replace=False
        )
    else:
        selected_products = all_products
    
    # Sous-échantillonner les données
    df_light = df[df['id'].isin(selected_products)].copy()
    
    # Réduire la période temporelle
    max_date = df_light['date'].max()
    min_date = max_date - pd.Timedelta(days=days_back)
    df_light = df_light[df_light['date'] >= min_date]
    
    print(f"✅ Dataset réduit: {len(df_light)} lignes au lieu de {len(df)}")
    print(f"✅ Produits: {len(selected_products)} au lieu de {len(all_products)}")
    
    return df_light

# Utilisation avec 12 mois et 20 produits
df_light = create_lightweight_dataset(df, max_products=20, days_back=365)

# Option 3: Garder toutes les données temporelles
df_light = create_lightweight_dataset(df, max_products=15, days_back=1941)

✅ Dataset réduit: 1810 lignes au lieu de 58327370
✅ Produits: 10 au lieu de 30490


In [23]:
## Sauvegarder le dataframe allégé
df_light.to_csv("data/df_light_teen_products.csv", index=False)


### Étape 1 : Création des séquences temporelles

In [24]:
df = pd.read_csv("data/df_light_teen_products.csv")

In [25]:
df_light.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,day_num,date,...,store_id_encoded,state_id_encoded,day_of_week,day_of_month,week_of_year,is_weekend,is_month_start,is_month_end,snap,has_event
52809166,HOBBIES_2_071_CA_1_validation,HOBBIES_2_071,HOBBIES_2,HOBBIES,CA_1,CA,d_1733,0,1733,2015-10-27,...,0,0,1,27,44,0,0,0,0,0
52811560,FOODS_3_656_CA_1_validation,FOODS_3_656,FOODS_3,FOODS,CA_1,CA,d_1733,0,1733,2015-10-27,...,0,0,1,27,44,0,0,0,0,0
52819024,HOUSEHOLD_2_101_CA_4_validation,HOUSEHOLD_2_101,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,d_1733,0,1733,2015-10-27,...,3,0,1,27,44,0,0,0,0,0
52820823,FOODS_3_775_CA_4_validation,FOODS_3_775,FOODS_3,FOODS,CA_4,CA,d_1733,0,1733,2015-10-27,...,3,0,1,27,44,0,0,0,0,0
52821019,HOBBIES_1_148_TX_1_validation,HOBBIES_1_148,HOBBIES_1,HOBBIES,TX_1,TX,d_1733,1,1733,2015-10-27,...,4,1,1,27,44,0,0,0,0,0


In [26]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Configuration des paramètres
SEQ_LENGTH = 56  # Fenêtre temporelle (8 semaines)
PREDICTION_HORIZON = 28  # Horizon de prédiction
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# Features numériques et catégorielles
numeric_features = ['sales', 'sell_price', 'day_of_week', 'day_of_month', 
                   'week_of_year', 'month', 'year', 'is_weekend', 'is_month_start', 
                   'is_month_end', 'snap', 'has_event']

categorical_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

### Étape 2 : Préprocessing et normalisation

In [28]:
# Conversion de la colonne date en datetime
df['date'] = pd.to_datetime(df['date'])

# Encodage des variables catégorielles
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df[f'{col}_encoded'] = le.fit_transform(df[col])
    label_encoders[col] = le

# Création des features temporelles
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_month'] = df['date'].dt.day
df['week_of_year'] = df['date'].dt.isocalendar().week
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
df['snap'] = df[['snap_CA', 'snap_TX', 'snap_WI']].max(axis=1)
df['has_event'] = df['event_name_1'].notna().astype(int)

# Remplir les valeurs manquantes (méthode mise à jour)
df['sell_price'] = df.groupby(['item_id', 'store_id'])['sell_price'].ffill().bfill()

# Normalisation des features numériques
scaler = StandardScaler()
df_numeric_scaled = pd.DataFrame(
    scaler.fit_transform(df[numeric_features]),
    columns=numeric_features,
    index=df.index
)

# DataFrame final pour le DL
df_dl = pd.concat([
    df_numeric_scaled,
    df[[f'{col}_encoded' for col in categorical_features]],
    df[['date', 'id']]
], axis=1)

print("✅ Données préparées pour le Deep Learning")
print(f"Shape: {df_dl.shape}")
df_dl.head()

✅ Données préparées pour le Deep Learning
Shape: (1810, 19)


,sales,sell_price,day_of_week,day_of_month,week_of_year,month,year,is_weekend,is_month_start,is_month_end,snap,has_event,item_id_encoded,dept_id_encoded,cat_id_encoded,store_id_encoded,state_id_encoded,date,id
0,-0.558550,-1.213621,-1.011822,1.28889,1.017858,0.974069,-1.320009,-0.634902,-0.185164,-0.185164,-0.99449,-0.3114,5,2,1,0,0,2015-10-27,HOBBIES_2_071_CA_1_validation
1,-0.558550,-0.491074,-1.011822,1.28889,1.017858,0.974069,-1.320009,-0.634902,-0.185164,-0.185164,-0.99449,-0.3114,2,0,0,0,0,2015-10-27,FOODS_3_656_CA_1_validation
2,-0.558550,-0.200329,-1.011822,1.28889,1.017858,0.974069,-1.320009,-0.634902,-0.185164,-0.185164,-0.99449,-0.3114,7,4,2,1,0,2015-10-27,HOUSEHOLD_2_101_CA_4_validation
3,-0.558550,-0.404714,-1.011822,1.28889,1.017858,0.974069,-1.320009,-0.634902,-0.185164,-0.185164,-0.99449,-0.3114,3,0,0,1,0,2015-10-27,FOODS_3_775_CA_4_validation
4,0.383645,-0.088060,-1.011822,1.28889,1.017858,0.974069,-1.320009,-0.634902,-0.185164,-0.185164,-0.99449,-0.3114,4,1,1,2,1,2015-10-27,HOBBIES_1_148_TX_1_validation


### Étape 3 : Création des séquences avec tf.data

In [29]:
def create_sequences_for_product(product_data, seq_length=SEQ_LENGTH, horizon=PREDICTION_HORIZON):
    """Crée des séquences pour un produit spécifique"""
    sequences = []
    targets = []
    
    # Features pour le modèle (sans les métadonnées)
    feature_columns = numeric_features + [f'{col}_encoded' for col in categorical_features]
    values = product_data[feature_columns].values
    
    # Création des séquences glissantes
    for i in range(len(values) - seq_length - horizon + 1):
        sequences.append(values[i:(i + seq_length)])
        targets.append(values[i + seq_length:i + seq_length + horizon, 0])  # colonne 0 = sales
    
    return np.array(sequences), np.array(targets)

def create_tf_dataset(sequences, targets, batch_size=BATCH_SIZE, shuffle=True):
    """Convertit en tf.data.Dataset optimisé"""
    dataset = tf.data.Dataset.from_tensor_slices((sequences, targets))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

### Étape 4 : Préparation des datasets complets

In [30]:
# Séparation des produits pour train/validation
product_ids = df_dl['id'].unique()
np.random.shuffle(product_ids)

val_size = int(len(product_ids) * VAL_SPLIT)
train_products = product_ids[val_size:]
val_products = product_ids[:val_size]

print(f"Produits d'entraînement: {len(train_products)}")
print(f"Produits de validation: {len(val_products)}")

# Construction des datasets
def build_datasets(product_list, df):
    all_sequences = []
    all_targets = []
    
    for product_id in product_list:
        product_data = df[df['id'] == product_id].sort_values('date')
        if len(product_data) >= SEQ_LENGTH + PREDICTION_HORIZON:
            sequences, targets = create_sequences_for_product(product_data)
            if len(sequences) > 0:
                all_sequences.append(sequences)
                all_targets.append(targets)
    
    # Concatenation de tous les produits
    all_sequences = np.concatenate(all_sequences, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    return all_sequences, all_targets

# Construction des datasets d'entraînement et validation
print("Construction du dataset d'entraînement...")
X_train, y_train = build_datasets(train_products, df_dl)

print("Construction du dataset de validation...")
X_val, y_val = build_datasets(val_products, df_dl)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

# Création des tf.data.Dataset
train_dataset = create_tf_dataset(X_train, y_train, shuffle=True)
val_dataset = create_tf_dataset(X_val, y_val, shuffle=False)

Produits d'entraînement: 8
Produits de validation: 2
Construction du dataset d'entraînement...
Construction du dataset de validation...
X_train shape: (784, 56, 17), y_train shape: (784, 28)
X_val shape: (196, 56, 17), y_val shape: (196, 28)


### Architectures de modèles séquentiels

#### Modèle 1 : LSTM Simple

In [31]:
def build_lstm_model(seq_length=SEQ_LENGTH, num_features=None):
    """LSTM simple avec plusieurs couches"""
    if num_features is None:
        num_features = X_train.shape[2]
    
    model = tf.keras.Sequential([
        # Couche LSTM avec return_sequences=True pour les couches suivantes
        tf.keras.layers.LSTM(128, return_sequences=True, 
                           input_shape=(seq_length, num_features)),
        tf.keras.layers.Dropout(0.2),
        
        # Deuxième couche LSTM
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        
        # Troisième couche LSTM
        tf.keras.layers.LSTM(32, return_sequences=False),
        tf.keras.layers.Dropout(0.2),
        
        # Couches denses finales
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(PREDICTION_HORIZON)  # 28 jours de prédiction
    ])
    
    return model

# Construction et compilation
lstm_model = build_lstm_model()
lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mape']
)

print("✅ Modèle LSTM construit")
lstm_model.summary()

c:\Users\0000\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Modèle LSTM construit


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 56, 128)        │        74,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 56, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 56, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 28)             │           924 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 141,692 (553.48 KB)

 Trainable params: 141,692 (553.48 KB)

 Non-trainable params: 0 (0.00 B)

#### Modèle 2 : GRU avec régularisation

In [32]:
def build_gru_model(seq_length=SEQ_LENGTH, num_features=None):
    """GRU avec régularisation avancée"""
    if num_features is None:
        num_features = X_train.shape[2]
    
    model = tf.keras.Sequential([
        # Couche GRU avec régularisation
        tf.keras.layers.GRU(128, return_sequences=True,
                          recurrent_dropout=0.1,
                          dropout=0.2,
                          input_shape=(seq_length, num_features)),
        
        # Batch Normalization
        tf.keras.layers.BatchNormalization(),
        
        # Deuxième couche GRU
        tf.keras.layers.GRU(64, return_sequences=False,
                          recurrent_dropout=0.1,
                          dropout=0.2),
        
        tf.keras.layers.BatchNormalization(),
        
        # Couches denses avec régularisation
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer='l2'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer='l2'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(PREDICTION_HORIZON)
    ])
    
    return model

gru_model = build_gru_model()
gru_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

#### Modèle 3 : CNN1D + LSTM (Hybride)

In [33]:
def build_cnn_lstm_model(seq_length=SEQ_LENGTH, num_features=None):
    """Architecture hybride CNN1D pour features locales + LSTM pour dépendances longues"""
    if num_features is None:
        num_features = X_train.shape[2]
    
    model = tf.keras.Sequential([
        # CNN 1D pour extraire les patterns locaux
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu',
                             input_shape=(seq_length, num_features)),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Dropout(0.2),
        
        # LSTM pour les dépendances temporelles longues
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.LSTM(32, return_sequences=False),
        tf.keras.layers.Dropout(0.2),
        
        # Couches denses finales
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(PREDICTION_HORIZON)
    ])
    
    return model

cnn_lstm_model = build_cnn_lstm_model()
cnn_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

c:\Users\0000\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Modèle 4 : Architecture Bidirectionnelle

In [34]:
def build_bidirectional_model(seq_length=SEQ_LENGTH, num_features=None):
    """LSTM bidirectionnel pour capturer le contexte passé et futur"""
    if num_features is None:
        num_features = X_train.shape[2]
    
    model = tf.keras.Sequential([
        # Couche bidirectionnelle
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(64, return_sequences=True),
            input_shape=(seq_length, num_features)
        ),
        tf.keras.layers.Dropout(0.2),
        
        # Deuxième couche bidirectionnelle
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(32, return_sequences=False)
        ),
        tf.keras.layers.Dropout(0.2),
        
        # Attention mechanism (simplifié)
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(PREDICTION_HORIZON)
    ])
    
    return model

bidirectional_model = build_bidirectional_model()
bidirectional_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

c:\Users\0000\anaconda3\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


###  Entraînement des modèles

#### Callbacks et configuration d'entraînement

In [35]:
# Callbacks pour l'entraînement
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False
)

# Configuration d'entraînement
training_config = {
    'epochs': 100,
    'batch_size': BATCH_SIZE,
    'callbacks': [early_stopping, reduce_lr, model_checkpoint],
    'validation_data': val_dataset
}

#### Entraînement du modèle LSTM

In [36]:
print("🎯 Entraînement du modèle LSTM...")
history_lstm = lstm_model.fit(
    train_dataset,
    epochs=training_config['epochs'],
    validation_data=val_dataset,
    callbacks=training_config['callbacks'],
    verbose=1
)

🎯 Entraînement du modèle LSTM...
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.6711 - mae: 0.6131 - mape: 90.4785

25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.6704 - mae: 0.6119 - mape: 90.2459 - val_loss: 2.1431 - val_mae: 1.0732 - val_mape: 97.6499 - learning_rate: 0.0010
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.5767 - mae: 0.5022 - mape: 71.8991

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.5775 - mae: 0.5024 - mape: 71.8723 - val_loss: 1.9905 - val_mae: 1.0216 - val_mape: 92.0682 - learning_rate: 0.0010
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5956 - mae: 0.4927 - mape: 68.9885 - val_loss: 1.9981 - val_mae: 1.0147 - val_mape: 89.4626 - learning_rate: 0.0010
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.5619 - mae: 0.4685 - mape: 65.6186

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.5621 - mae: 0.4688 - mape: 65.6638 - val_loss: 1.9648 - val_mae: 1.0054 - val_mape: 88.6164 - learning_rate: 0.0010
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5524 - mae: 0.4609 - mape: 64.7417 - val_loss: 1.9855 - val_mae: 1.0141 - val_mape: 89.9793 - learning_rate: 0.0010
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.5663 - mae: 0.4679 - mape: 64.8364

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.5660 - mae: 0.4679 - mape: 64.8621 - val_loss: 1.9238 - val_mae: 0.9963 - val_mape: 88.9424 - learning_rate: 0.0010
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.5638 - mae: 0.4711 - mape: 65.8546 - val_loss: 1.9556 - val_mae: 1.0046 - val_mape: 88.9075 - learning_rate: 0.0010
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5454 - mae: 0.4623 - mape: 64.9711 - val_loss: 1.9437 - val_mae: 0.9989 - val_mape: 87.8998 - learning_rate: 0.0010
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.5686 - mae: 0.4669 - mape: 64.3630

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.5681 - mae: 0.4668 - mape: 64.3641 - val_loss: 1.8962 - val_mae: 0.9958 - val_mape: 89.3266 - learning_rate: 0.0010
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5841 - mae: 0.4868 - mape: 68.2380 - val_loss: 1.9176 - val_mae: 1.0000 - val_mape: 89.3271 - learning_rate: 0.0010
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5098 - mae: 0.4363 - mape: 62.2171 - val_loss: 1.9255 - val_mae: 1.0055 - val_mape: 90.3231 - learning_rate: 0.0010
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5699 - mae: 0.4702 - mape: 65.3780 - val_loss: 1.9388 - val_mae: 1.0031 - val_mape: 89.5345 - learning_rate: 0.0010
Epoch 13/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5262 - mae: 0.4485 - mape: 63.5351 - val_loss: 1.9427 - val_mae: 1.0087 - val_mape: 90.4890 - learning_rate: 0.0010
Epoch 14/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.5469 - mae: 0.4664 - mape: 66.0561 - val_loss: 1.9342 -

#### Visualisation des résultats

#### Entrainement de GRU

In [39]:
print("🎯 Entraînement du modèle GRU...")
history_gru = gru_model.fit(
    train_dataset,
    epochs=training_config['epochs'],
    validation_data=val_dataset,
    callbacks=training_config['callbacks'],
    verbose=1
)   


🎯 Entraînement du modèle GRU...
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.5792 - mae: 0.4962 - val_loss: 1.9856 - val_mae: 1.0150 - learning_rate: 2.5000e-04
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.6283 - mae: 0.5116 - val_loss: 1.9378 - val_mae: 1.0019 - learning_rate: 2.5000e-04
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.6113 - mae: 0.4999 - val_loss: 1.9747 - val_mae: 1.0098 - learning_rate: 2.5000e-04
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.5798 - mae: 0.4895 - val_loss: 1.9543 - val_mae: 1.0062 - learning_rate: 2.5000e-04
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.5950 - mae: 0.4964 - val_loss: 1.9332 - val_mae: 1.0033 - learning_rate: 2.5000e-04
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.5925 - mae: 0.5005 - val_loss: 1.9643 - val_mae: 1.0080 - learning_rate: 2.5000e-04
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.6094 - mae: 0.5027 - val_loss:

#### Visualisation des résultats

In [41]:
print("🎯 Entraînement du modèle LSTM + CNN1D..")
history_lstm_cnn = cnn_lstm_model.fit(
    train_dataset,
    epochs=training_config['epochs'],
    validation_data=val_dataset,
    callbacks=training_config['callbacks'],
    verbose=1
)

🎯 Entraînement du modèle LSTM + CNN1D..
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.7085 - mae: 0.6608 - val_loss: 2.2336 - val_mae: 1.0942 - learning_rate: 0.0010
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6455 - mae: 0.5550 - val_loss: 2.1259 - val_mae: 1.0662 - learning_rate: 0.0010
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6218 - mae: 0.5113 - val_loss: 2.0754 - val_mae: 1.0418 - learning_rate: 0.0010
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6275 - mae: 0.5098 - val_loss: 2.0041 - val_mae: 1.0187 - learning_rate: 0.0010
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5791 - mae: 0.4938 - val_loss: 1.9682 - val_mae: 1.0004 - learning_rate: 0.0010
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5915 - mae: 0.4851 - val_loss: 1.9690 - val_mae: 1.0026 - learning_rate: 0.0010
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5679 - mae: 0.4823 - val_loss: 1.9637 - val_mae: 0.9

In [45]:
# Fonction de visualisation des prédictions
def visualize_predictions(model, X_val, y_val, scaler, num_samples=5):
    """
    Visualise les prédictions vs valeurs réelles pour plusieurs exemples
    """
    # Faire des prédictions
    predictions = model.predict(X_val[:num_samples])
    
    # Créer les graphiques
    fig, axes = plt.subplots(num_samples, 1, figsize=(15, 4*num_samples))
    if num_samples == 1:
        axes = [axes]
    
    for i in range(num_samples):
        ax = axes[i]
        
        # Préparer les données
        actual = y_val[i]
        predicted = predictions[i]
        days = range(1, len(actual) + 1)
        
        # Tracer les courbes
        ax.plot(days, actual, 'b-o', label='Valeurs réelles', linewidth=2, markersize=4)
        ax.plot(days, predicted, 'r--s', label='Prédictions', linewidth=2, markersize=4)
        
        # Calculer les métriques
        mae = np.mean(np.abs(actual - predicted))
        rmse = np.sqrt(np.mean((actual - predicted)**2))
        
        ax.set_title(f'Exemple {i+1} - MAE: {mae:.3f}, RMSE: {rmse:.3f}')
        ax.set_xlabel('Jours de prédiction')
        ax.set_ylabel('Ventes (normalisées)')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# # Visualiser les prédictions du modèle LSTM
# print("📊 Visualisation des prédictions LSTM")
# visualize_predictions(lstm_model, X_val, y_val, scaler, num_samples=5)
